In [1]:
import pandas as pd

In [2]:
blocks = pd.read_csv('https://github.com/UBICenter/local-child-allowance/raw/main/data/master_block.csv.gz')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
blocks = blocks[blocks['state_fip'] == 6]

In [4]:
upper_data = blocks.groupby(["puma", "upper_leg_district"]).population.sum()
upper_data = pd.DataFrame.from_dict(upper_data)
upper_data = upper_data.rename(columns={"population": "upper_district_population"})
upper_data = upper_data.reset_index()

In [5]:
lower_data = blocks.groupby(["puma", "lower_leg_district"]).population.sum()
lower_data = pd.DataFrame.from_dict(lower_data)
lower_data = lower_data.rename(columns={"population": "lower_district_population"})
lower_data = lower_data.reset_index()

In [6]:
puma_pops = blocks.groupby(["puma"]).population.sum()
puma_pops = pd.DataFrame.from_dict(puma_pops)
puma_pops = puma_pops.rename(columns={"population": "puma_population"})
puma_pops = puma_pops.reset_index()

In [7]:
sum = pd.merge(upper_data, puma_pops, on="puma")
sum['upper_pop_share_of_puma'] = sum.upper_district_population / sum.puma_population

In [8]:
sum2 = pd.merge(lower_data, puma_pops, on="puma")
sum2['lower_pop_share_of_puma'] = sum2.lower_district_population / sum2.puma_population

In [9]:
#sum = pd.merge(sum, sum2, on='puma')

In [10]:
person = pd.read_stata(
    "https://www2.census.gov/programs-surveys/supplemental-poverty-measure/datasets/spm/spm_2019_pu.dta",
    columns=[
        "serialno",
        "sporder",
        "wt",
        "age",
        "spm_id",
        "spm_povthreshold",
        "spm_resources",
        "st",
        "puma"
    ],
)

In [11]:
# Cleanup
person.columns = person.columns.str.lower()
person = person.rename(columns={'serialno': 'serial', 'sporder':'pernum'})

In [12]:
person = person.astype({"serial":'int', "pernum":'int',
                          "wt":'int', "age":'int',
                          "spm_id":'int', "spm_povthreshold":'int',
                          "spm_resources": "int"})

In [13]:
# Sort to just Maryland
person = person[person['st'] == 6]

In [14]:
person['old_puma'] = person['puma'].apply(str)
person = person.drop(['puma'], axis=1)

In [15]:
person["puma"] = person.old_puma.str[3:6]

In [16]:
person['puma'] = person['puma'].apply(int)

In [17]:
person_with_upper = person.merge(sum, on="puma")
person_with_upper['adj_wt'] = person_with_upper.wt * person_with_upper.upper_pop_share_of_puma

In [18]:
person_with_lower = person.merge(sum2, on="puma")
person_with_lower['adj_wt'] = person_with_lower.wt * person_with_lower.lower_pop_share_of_puma

In [19]:
person_with_upper

,serial,pernum,wt,age,spm_id,spm_povthreshold,spm_resources,st,old_puma,puma,upper_leg_district,upper_district_population,puma_population,upper_pop_share_of_puma,adj_wt
0,26,1,88,21,26001,41353,27386,6.0,608105.0,105,9,164900.950871,174474.0,0.945132,83.171611
1,26,1,88,21,26001,41353,27386,6.0,608105.0,105,10,9573.049129,174474.0,0.054868,4.828389
2,26,2,106,4,26001,41353,27386,6.0,608105.0,105,9,164900.950871,174474.0,0.945132,100.183986
3,26,2,106,4,26001,41353,27386,6.0,608105.0,105,10,9573.049129,174474.0,0.054868,5.816014
4,26,3,106,2,26001,41353,27386,6.0,608105.0,105,9,164900.950871,174474.0,0.945132,100.183986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99533,1407032,3,119,11,1407032001,25929,29551,6.0,607107.0,107,10,159856.900444,159937.0,0.999499,118.940402
99534,1410672,1,116,30,1410672001,22128,69785,6.0,607107.0,107,7,80.099556,159937.0,0.000501,0.058095
99535,1410672,1,116,30,1410672001,22128,69785,6.0,607107.0,107,10,159856.900444,159937.0,0.999499,115.941905
99536,1410672,2,146,4,1410672001,22128,69785,6.0,607107.0,107,7,80.099556,159937.0,0.000501,0.073120


In [20]:
person_with_lower

,serial,pernum,wt,age,spm_id,spm_povthreshold,spm_resources,st,old_puma,puma,lower_leg_district,lower_district_population,puma_population,lower_pop_share_of_puma,adj_wt
0,26,1,88,21,26001,41353,27386,6.0,608105.0,105,15,0.0,174474.0,0.000000,0.000000
1,26,1,88,21,26001,41353,27386,6.0,608105.0,105,18,172946.0,174474.0,0.991242,87.229318
2,26,1,88,21,26001,41353,27386,6.0,608105.0,105,20,1528.0,174474.0,0.008758,0.770682
3,26,1,88,21,26001,41353,27386,6.0,608105.0,105,25,0.0,174474.0,0.000000,0.000000
4,26,2,106,4,26001,41353,27386,6.0,608105.0,105,15,0.0,174474.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150198,1407032,1,116,42,1407032001,25929,29551,6.0,607107.0,107,20,159937.0,159937.0,1.000000,116.000000
150199,1407032,2,124,14,1407032001,25929,29551,6.0,607107.0,107,20,159937.0,159937.0,1.000000,124.000000
150200,1407032,3,119,11,1407032001,25929,29551,6.0,607107.0,107,20,159937.0,159937.0,1.000000,119.000000
150201,1410672,1,116,30,1410672001,22128,69785,6.0,607107.0,107,20,159937.0,159937.0,1.000000,116.000000


In [21]:
person_with_upper.to_csv('CA_ACS_with_Upper_Districts_2010.csv')

In [22]:
person_with_lower.to_csv('CA_ACS_with_Lower_Districts_2010.csv')